# This file is used to train

In [17]:
import torch
import numpy as np
from tqdm import tqdm
import pandas as pd
from sc_model import SC_LSTM as Model

# Device init

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    !nvidia-smi
    print(torch.cuda.get_device_name(0))

else:
    print("No GPU :(")

Sun Jul  9 03:18:48 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.41.03              Driver Version: 530.41.03    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti      Off| 00000000:01:00.0  On |                  N/A |
| N/A   50C    P0               20W /  N/A|    741MiB /  4096MiB |     19%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [19]:
device = 'cpu'

## Loading the data

In [20]:
from Data.PreprocessorOfGeneratedData import Preprocessor

preproc = Preprocessor()

data = pd.read_csv('../Data/schedule_gen.csv')
data

,Label Number,Duration,Importance,Start Time,Date
0,3,40,2,09:26,10/07/2023
1,1,50,0,10:31,10/07/2023
2,2,20,0,12:41,10/07/2023
3,0,15,0,14:01,10/07/2023
4,2,20,1,16:16,10/07/2023
...,...,...,...,...,...
707,2,90,0,08:45,17/10/2023
708,1,20,0,13:00,17/10/2023
709,3,170,2,16:05,17/10/2023
710,1,40,2,20:15,17/10/2023


In [21]:
input_data, type_vector, output_vector = preproc.preprocess('../Data/schedule_gen.csv')

In [22]:
input_data

,Label Number_0,Label Number_1,Label Number_2,Label Number_3,Duration,Importance_0,Importance_1,Importance_2,Importance_3,Time_Min,...,Date_Month_sin,Date_Month_cos,Plan_Time_Min,Plan_Time_Min_sin,Plan_Time_Min_cos,Plan_Date_Categorical,Plan_Date_Day_sin,Plan_Date_Day_cos,Plan_Date_Month_sin,Plan_Date_Month_cos
0,0,0,0,0,0.212121,0,0,0,0,0.393056,...,-0.500000,-0.866025,0.385417,0.659346,-0.751840,0.523288,0.897805,-0.440394,-0.500000,-0.866025
1,0,0,0,0,0.272727,0,1,0,0,0.438194,...,-0.500000,-0.866025,0.388889,0.642788,-0.766044,0.523288,0.897805,-0.440394,-0.500000,-0.866025
2,0,0,0,0,0.090909,0,0,0,0,0.528472,...,-0.500000,-0.866025,0.388889,0.642788,-0.766044,0.523288,0.897805,-0.440394,-0.500000,-0.866025
3,0,0,0,1,0.060606,0,0,0,0,0.584028,...,-0.500000,-0.866025,0.388889,0.642788,-0.766044,0.523288,0.897805,-0.440394,-0.500000,-0.866025
4,0,0,0,0,0.090909,0,0,0,0,0.000000,...,-0.500000,-0.866025,0.388889,0.642788,-0.766044,0.523288,0.897805,-0.440394,-0.500000,-0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
707,0,0,0,0,0.515152,0,0,0,0,0.364583,...,-0.866025,0.500000,0.358333,0.777146,-0.629320,0.794521,-0.299363,-0.954139,-0.866025,0.500000
708,0,0,0,0,0.090909,0,0,0,0,0.541667,...,0.000000,1.000000,0.538194,-0.237686,-0.971342,0.794521,-0.299363,-0.954139,-0.866025,0.500000
709,0,0,0,0,1.000000,0,0,0,0,0.670139,...,-0.866025,0.500000,0.667361,-0.868199,-0.496217,0.794521,-0.299363,-0.954139,-0.866025,0.500000
710,0,0,0,0,0.212121,0,0,0,0,0.843750,...,-0.866025,0.500000,0.667361,-0.868199,-0.496217,0.794521,-0.299363,-0.954139,-0.866025,0.500000


In [23]:
output_vector

,start,end,refr
0,1.273611,0.027778,0.0
1,1.31875,0.034722,0.0
2,1.409028,0.013889,0.0
3,1.464583,0.010417,0.0
4,0.880556,0.013889,0.0
...,...,...,...
707,100.245139,0.0625,0.0
708,100.422222,0.013889,0.0
709,100.550694,0.118056,0.0
710,100.724306,0.027778,0.0


In [24]:
# Make all output values positive
output_vector = np.abs(output_vector)

# Add first element as 1440, 1440, 1440 to the output vector
output_vector = np.insert(output_vector, 0, [alpha]*3, axis=0)

pd.DataFrame(output_vector)

NameError: name 'alpha' is not defined

In [ ]:
from sklearn.preprocessing import MinMaxScaler

in_scaler = MinMaxScaler()
input_data = in_scaler.fit_transform(input_data)

out_scaler = MinMaxScaler()
output_vector = out_scaler.fit_transform(output_vector)

# Remove first element from the output vector
output_vector = np.delete(output_vector, 0, axis=0)

In [ ]:
# TODO DELETE
# print the output vector as pandas dataframe
pd.DataFrame(output_vector)

## Creating the dataset

In [ ]:
from torch.utils.data import Dataset, DataLoader

class ScheduleDataset(Dataset):
    def __init__(self, input_data, type_vector, output_vector, task_types='all', transform=None):
        if task_types == 'all':
            self.input_data = input_data
            self.type_vector = type_vector
            self.output_vector = output_vector
        elif task_types == 'resched':
            self.input_data = []
            self.type_vector = []
            self.output_vector = []
            for i in range(len(input_data)):
                if type_vector[i] == "resched":
                    self.input_data.append(input_data[i])
                    self.type_vector.append(type_vector[i])
                    self.output_vector.append(output_vector[i])
        elif task_types == 'non-resched':
            self.input_data = []
            self.type_vector = []
            self.output_vector = []
            for i in range(len(input_data)):
                if type_vector[i] == "non-resched":
                    self.input_data.append(input_data[i])
                    self.type_vector.append(type_vector[i])
                    self.output_vector.append(output_vector[i])
        else:
            raise ValueError("task_types must be 'all', 'resched', or 'non-resched'")
        self.transform = transform

    def __len__(self):
        return len(self.input_data)

    def __getitem__(self, idx):
        return self.input_data[idx], self.type_vector[idx], self.output_vector[idx]


In [ ]:
resched_dataset = ScheduleDataset(input_data, type_vector, output_vector, task_types='resched')
non_resched_dataset = ScheduleDataset(input_data, type_vector, output_vector, task_types='non-resched')
all_dataset = ScheduleDataset(input_data, type_vector, output_vector, task_types='all')

print(f"Resched dataset size: {len(resched_dataset)}\n"
      f"Non-resched dataset size: {len(non_resched_dataset)}\n"
      f"All dataset size: {len(all_dataset)}")

## Create dataloaders

In [ ]:
batch_size = 1
# resched_dataset_size = len(resched_dataset)
# train_size = int(resched_dataset_size * 0.8)
# test_size = resched_dataset_size - train_size
#
# train_resched_dataset, test_resched_dataset = torch.utils.data.random_split(resched_dataset, [train_size, test_size])

train_resched_dataloader = DataLoader(resched_dataset, batch_size=batch_size)
# valid_resched_dataloader = DataLoader2(resched_dataset, batch_size=batch_size)

train_non_resched_dataloader = DataLoader(non_resched_dataset, batch_size=batch_size)

## Create free time slots generator

In [ ]:
from Data.GeneratorOfAvailableTimeslots import GeneratorOfAvailableTimeslots

time_slots_gen = GeneratorOfAvailableTimeslots(5)
time_slots = time_slots_gen.generate_available_timeslots()

time_slots      # TODO: why all the intervals are closed?
single_interval = time_slots[0]
single_interval

## Init the model

In [ ]:
# check if the dataloader works and get the input and output sizes
in_features = 22
out_features = 3
for i, (features, a, ans) in enumerate(train_resched_dataloader):
    in_features = features.shape[1]
    out_features = ans.shape[1]
    break

n_layers = 1
hidden_size = 124
hidden_injector = 64

# Create the model
SC_LSTM = Model(in_features, n_layers, hidden_size, out_features, batch_size, hidden_injector=hidden_injector).to(device)

# Training the model

In [ ]:
# Configure hyper-parameters
epochs = 20
learning_rate = 0.001
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(SC_LSTM.parameters(), lr=learning_rate)

history = []
loss_accomulator = []
mean_loss = None

In [ ]:
# Firstly train the LSTM using only reschedulable tasks
SC_LSTM.train_model(mode='lstm')

for epoch in range(0, epochs):
    SC_LSTM.reset_states()
    for i, (X, task_type, Y) in enumerate(tqdm(train_resched_dataloader, desc=f"Epoch: {epoch + 1}, Mean Loss: {mean_loss}", leave=False, colour='green')):

        # Convert X and Y to the correct type
        X = torch.Tensor(X).type(torch.float32).to(device)
        Y = torch.Tensor(Y).type(torch.float32).to(device)

        # Make prediction
        Y_pred = SC_LSTM.forward(X, task_type=task_type, free_time_slots=single_interval, save_states=True)

        # Calculate loss
        loss = loss_func(Y_pred.view(-1, ), Y.view(-1, ))

        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Append loss to list
        loss_accomulator.append(loss.item())

    mean_loss = np.mean(loss_accomulator)
    history.append(mean_loss)
    loss_accomulator = []

In [ ]:
import matplotlib.pyplot as plt
# Plot the loss history
plt.plot(history)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss history")
plt.show()

In [ ]:
# Configure hyper-parameters
epochs = 20
learning_rate = 0.001
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(SC_LSTM.parameters(), lr=learning_rate)

history = []
loss_accomulator = []

In [ ]:
# Now train the injector using non-reschedulable tasks
SC_LSTM.train_model(mode='injector')

for epoch in range(0, epochs):
    SC_LSTM.reset_states()

    for i, (X, task_type, Y) in enumerate(tqdm(train_non_resched_dataloader, desc=f"Epoch: {epoch + 1}, Mean Loss: {mean_loss}", leave=False, colour='green')):

        # Convert X and Y to the correct type
        X = torch.Tensor(X).type(torch.float32).to(device)
        Y = torch.Tensor(Y).type(torch.float32).to(device)

        # Make prediction
        Y_pred = SC_LSTM.forward(X, task_type=task_type, free_time_slots=single_interval, save_states=True)

        # Calculate loss
        loss = loss_func(Y_pred.view(-1, ), Y.view(-1, ))

        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Append loss to list
        loss_accomulator.append(loss.item())

    mean_loss = np.mean(loss_accomulator)
    history.append(mean_loss)
    loss_accomulator = []

In [ ]:
# Plot the loss history
plt.plot(history)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss history")
plt.show()

# Save weights

In [ ]:
torch.save(SC_LSTM.state_dict(), "sc_lstm_weights.pth")